In [1]:
from __future__ import print_function
import sys
from random import random
from operator import add
from pyspark.sql import SparkSession
import os
import pyspark.sql.functions as f
  
spark = SparkSession.builder.appName("PyPi").getOrCreate()
df_all = spark.read.option('lineSep', r'(THE\sEND)').text("./data/Shakespeare.txt")

23/01/26 09:55:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = df_all.withColumn('value', f.regexp_replace('value', r'<<[\w\s\d\n()\,.-]{495}>>', ''))\
            .withColumn('value', f.explode(f.split('value', r'THE\sEND', -1))) \
            .withColumn('index', f.monotonically_increasing_id())\
            .filter("index > 1")\
            .filter('index < 38') \
            .withColumn("title", f.regexp_extract('value', r'(.*)\n*by', 0))\
            .withColumn("value", f.regexp_replace('value', r'([A-Z ,]*)\n*by William Shakespeare', ''))\
            .withColumn("title", f.regexp_replace('title', r'\n*by', ''))\
            .withColumn("year", f.regexp_extract('value', r'\d{4}', 0))\
            .withColumn("value", f.regexp_replace('value', r'\d{4}', ''))\
            .withColumn('value', f.trim('value'))\
            .withColumn('value', f.regexp_replace('value', r' {2,}', ' '))\
            .withColumn('value', f.regexp_replace('value', r'\n{2,}', ''))\
            .withColumn('wordCount', f.size(f.split('value', ' ')))\
            .withColumn('lineCount', f.size(f.split('value', r'\n')))\
            .orderBy(f.col("lineCount").desc())

In [4]:
def play_counts(df):
    results = df.select('title','lineCount', 'wordCount').collect()
    for r in results:
        print(f"{r['title']}, {r['lineCount']} lines, {r['wordCount']} words.")

# df.filter("index == 37").collect()
play_counts(df)

THE TRAGEDY OF HAMLET, PRINCE OF DENMARK, 3947 lines, 32079 words.
KING RICHARD III, 3914 lines, 31193 words.
THE TRAGEDY OF CORIOLANUS, 3691 lines, 29293 words.
CYMBELINE, 3649 lines, 28870 words.
THE TRAGEDY OF ANTONY AND CLEOPATRA, 3587 lines, 26552 words.
THE TRAGEDY OF OTHELLO, MOOR OF VENICE, 3479 lines, 27986 words.
THE TRAGEDY OF KING LEAR, 3433 lines, 27585 words.
THE HISTORY OF TROILUS AND CRESSIDA, 3431 lines, 27623 words.
KING HENRY THE EIGHTH, 3327 lines, 25886 words.
THE WINTER'S TALE, 3249 lines, 26059 words.
THE LIFE OF KING HENRY THE FIFTH, 3147 lines, 27498 words.
THE SECOND PART OF KING HENRY THE SIXTH, 3133 lines, 26840 words.
SECOND PART OF KING HENRY IV, 3101 lines, 27689 words.
THE TRAGEDY OF ROMEO AND JULIET, 3089 lines, 25857 words.
THE THIRD PART OF KING HENRY THE SIXTH, 3012 lines, 25873 words.
THE FIRST PART OF KING HENRY THE FOURTH, 2926 lines, 25783 words.
THE FIRST PART OF HENRY THE SIXTH, 2852 lines, 22883 words.
KING RICHARD THE SECOND, 2851 lines, 2336